# 10-714 Project: Needle-DAGMM

Needle (NEcessary Elements of Deep Learning) Implementation for the Deep Autoencoding Gaussian Mixture Model (DAGMM)

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir -p 10714
%cd /content/drive/MyDrive/10714/project
!pip3 install pybind11

Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/10714/project
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 222 kB 5.0 MB/s 


In [2]:
!make

-- Found pybind11: /usr/local/lib/python3.8/dist-packages/pybind11/include (found version "2.10.2")
-- Found cuda, building cuda backend
Fri Dec 23 22:51:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    32W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+------------------------------

In [3]:
import sys
sys.path.append('./python')

In [4]:
!python3 -m pytest -l -v -k "op_squeeze"

============================= test session starts ==============================
platform linux -- Python 3.8.16, pytest-3.6.4, py-1.11.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/project, inifile:
plugins: typeguard-2.7.1
collected 104 items / 88 deselected                                            

tests/test_cholesky.py::test_op_squeeze[forward-cpu-x_shape0-None] PASSED [  6%]
tests/test_cholesky.py::test_op_squeeze[forward-cpu-x_shape1-0] PASSED   [ 12%]
tests/test_cholesky.py::test_op_squeeze[forward-cpu-x_shape2-1] PASSED   [ 18%]
tests/test_cholesky.py::test_op_squeeze[forward-cpu-x_shape3-1] PASSED   [ 25%]
tests/test_cholesky.py::test_op_squeeze[forward-cuda-x_shape0-None] PASSED [ 31%]
tests/test_cholesky.py::test_op_squeeze[forward-cuda-x_shape1-0] PASSED  [ 37%]
tests/test_cholesky.py::test_op_squeeze[forward-cuda-x_shape2-1] PASSED  [ 43%]
tests/test_cholesky.py::test_op_squeeze[forward-cuda-x_shape3-1] PASSED  [ 50%]

In [5]:
!python3 -m pytest -l -v -k "op_unsqueeze"

============================= test session starts ==============================
platform linux -- Python 3.8.16, pytest-3.6.4, py-1.11.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/project, inifile:
plugins: typeguard-2.7.1
collected 104 items / 84 deselected                                            

tests/test_cholesky.py::test_op_unsqueeze[forward-cpu-x_shape0-0] PASSED [  5%]
tests/test_cholesky.py::test_op_unsqueeze[forward-cpu-x_shape1-1] PASSED [ 10%]
tests/test_cholesky.py::test_op_unsqueeze[forward-cpu-x_shape2-0] PASSED [ 15%]
tests/test_cholesky.py::test_op_unsqueeze[forward-cpu-x_shape3-1] PASSED [ 20%]
tests/test_cholesky.py::test_op_unsqueeze[forward-cpu-x_shape4-2] PASSED [ 25%]
tests/test_cholesky.py::test_op_unsqueeze[forward-cuda-x_shape0-0] PASSED [ 30%]
tests/test_cholesky.py::test_op_unsqueeze[forward-cuda-x_shape1-1] PASSED [ 35%]
tests/test_cholesky.py::test_op_unsqueeze[forward-cuda-x_shape2-0] PASSED [ 40%]

In [7]:
!python3 -m pytest -l -v -k "op_norm"

============================= test session starts ==============================
platform linux -- Python 3.8.16, pytest-3.6.4, py-1.11.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/project, inifile:
plugins: typeguard-2.7.1
collected 104 items / 92 deselected                                            

tests/test_cholesky.py::test_op_norm[forward-cpu-x_shape0-0] PASSED      [  8%]
tests/test_cholesky.py::test_op_norm[forward-cpu-x_shape1-0] PASSED      [ 16%]
tests/test_cholesky.py::test_op_norm[forward-cpu-x_shape2-1] PASSED      [ 25%]
tests/test_cholesky.py::test_op_norm[forward-cuda-x_shape0-0] PASSED     [ 33%]
tests/test_cholesky.py::test_op_norm[forward-cuda-x_shape1-0] PASSED     [ 41%]
tests/test_cholesky.py::test_op_norm[forward-cuda-x_shape2-1] PASSED     [ 50%]
tests/test_cholesky.py::test_op_norm[backward-cpu-x_shape0-0] PASSED     [ 58%]
tests/test_cholesky.py::test_op_norm[backward-cpu-x_shape1-0] PASSED     [ 66%]
te

In [8]:
!python3 -m pytest -l -v -k "op_cosine_similarity"

============================= test session starts ==============================
platform linux -- Python 3.8.16, pytest-3.6.4, py-1.11.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/project, inifile:
plugins: typeguard-2.7.1
collected 104 items / 92 deselected                                            

tests/test_cholesky.py::test_op_cosine_similarity[forward-cpu-x_shape0-0] PASSED [  8%]
tests/test_cholesky.py::test_op_cosine_similarity[forward-cpu-x_shape1-0] PASSED [ 16%]
tests/test_cholesky.py::test_op_cosine_similarity[forward-cpu-x_shape2-1] PASSED [ 25%]
tests/test_cholesky.py::test_op_cosine_similarity[forward-cuda-x_shape0-0] PASSED [ 33%]
tests/test_cholesky.py::test_op_cosine_similarity[forward-cuda-x_shape1-0] PASSED [ 41%]
tests/test_cholesky.py::test_op_cosine_similarity[forward-cuda-x_shape2-1] PASSED [ 50%]
tests/test_cholesky.py::test_op_cosine_similarity[backward-cpu-x_shape0-0] PASSED [ 58%]
tests/test_cholesky.py

In [13]:
!python3 -m pytest -l -v -k "op_pairwise_distance"

============================= test session starts ==============================
platform linux -- Python 3.8.16, pytest-3.6.4, py-1.11.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/project, inifile:
plugins: typeguard-2.7.1
collected 100 items / 92 deselected                                            

tests/test_cholesky.py::test_op_pairwise_distance[forward-cpu-x_shape0-0] PASSED [ 12%]
tests/test_cholesky.py::test_op_pairwise_distance[forward-cpu-x_shape1-1] PASSED [ 25%]
tests/test_cholesky.py::test_op_pairwise_distance[forward-cuda-x_shape0-0] PASSED [ 37%]
tests/test_cholesky.py::test_op_pairwise_distance[forward-cuda-x_shape1-1] PASSED [ 50%]
tests/test_cholesky.py::test_op_pairwise_distance[backward-cpu-x_shape0-0] PASSED [ 62%]
tests/test_cholesky.py::test_op_pairwise_distance[backward-cpu-x_shape1-1] PASSED [ 75%]
tests/test_cholesky.py::test_op_pairwise_distance[backward-cuda-x_shape0-0] PASSED [ 87%]
tests/test_cholesky.

In [10]:
!python3 -m pytest -l -v -k "op_softmax"

============================= test session starts ==============================
platform linux -- Python 3.8.16, pytest-3.6.4, py-1.11.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/project, inifile:
plugins: typeguard-2.7.1
collected 104 items / 92 deselected                                            

tests/test_cholesky.py::test_op_softmax[forward-cpu-x_shape0-0] PASSED   [  8%]
tests/test_cholesky.py::test_op_softmax[forward-cpu-x_shape1-1] PASSED   [ 16%]
tests/test_cholesky.py::test_op_softmax[forward-cpu-x_shape2-1] PASSED   [ 25%]
tests/test_cholesky.py::test_op_softmax[forward-cuda-x_shape0-0] PASSED  [ 33%]
tests/test_cholesky.py::test_op_softmax[forward-cuda-x_shape1-1] PASSED  [ 41%]
tests/test_cholesky.py::test_op_softmax[forward-cuda-x_shape2-1] PASSED  [ 50%]
tests/test_cholesky.py::test_op_softmax[backward-cpu-x_shape0-0] PASSED  [ 58%]
tests/test_cholesky.py::test_op_softmax[backward-cpu-x_shape1-1] PASSED  [ 66%]
te

In [11]:
!python3 -m pytest -l -v -k "op_bmm"

============================= test session starts ==============================
platform linux -- Python 3.8.16, pytest-3.6.4, py-1.11.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/project, inifile:
plugins: typeguard-2.7.1
collected 104 items / 96 deselected                                            

tests/test_cholesky.py::test_op_bmm[forward-cpu-A_shape0-B_shape0] PASSED [ 12%]
tests/test_cholesky.py::test_op_bmm[forward-cpu-A_shape1-B_shape1] PASSED [ 25%]
tests/test_cholesky.py::test_op_bmm[forward-cuda-A_shape0-B_shape0] PASSED [ 37%]
tests/test_cholesky.py::test_op_bmm[forward-cuda-A_shape1-B_shape1] PASSED [ 50%]
tests/test_cholesky.py::test_op_bmm[backward-cpu-A_shape0-B_shape0] PASSED [ 62%]
tests/test_cholesky.py::test_op_bmm[backward-cpu-A_shape1-B_shape1] PASSED [ 75%]
tests/test_cholesky.py::test_op_bmm[backward-cuda-A_shape0-B_shape0] PASSED [ 87%]
tests/test_cholesky.py::test_op_bmm[backward-cuda-A_shape1-B_shape1] 

In [12]:
!python3 -m pytest -l -v -k "dagmm"

============================= test session starts ==============================
platform linux -- Python 3.8.16, pytest-3.6.4, py-1.11.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/project, inifile:
plugins: typeguard-2.7.1
collected 104 items / 102 deselected                                           

tests/test_dagmm.py::test_dagmm[device0-2-120-3-4] PASSED                [ 50%]
tests/test_dagmm.py::test_dagmm[device1-2-120-3-4] PASSED                [100%]

=================== 2 passed, 102 deselected in 3.79 seconds ===================


In [5]:
!python3 -m pytest -l -v -k "op_inv"

============================= test session starts ==============================
platform linux -- Python 3.8.16, pytest-3.6.4, py-1.11.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/project, inifile:
plugins: typeguard-2.7.1
collected 116 items / 108 deselected                                           

tests/test_cholesky.py::test_op_inv[forward-cpu-A_shape0] PASSED         [ 12%]
tests/test_cholesky.py::test_op_inv[forward-cpu-A_shape1] PASSED         [ 25%]
tests/test_cholesky.py::test_op_inv[forward-cuda-A_shape0] PASSED        [ 37%]
tests/test_cholesky.py::test_op_inv[forward-cuda-A_shape1] PASSED        [ 50%]
tests/test_cholesky.py::test_op_inv[backward-cpu-A_shape0] PASSED        [ 62%]
tests/test_cholesky.py::test_op_inv[backward-cpu-A_shape1] PASSED        [ 75%]
tests/test_cholesky.py::test_op_inv[backward-cuda-A_shape0] PASSED       [ 87%]
tests/test_cholesky.py::test_op_inv[backward-cuda-A_shape1] PASSED       [100%]

=

In [10]:
!python3 -m pytest -l -v -k "op_det"

============================= test session starts ==============================
platform linux -- Python 3.8.16, pytest-3.6.4, py-1.11.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/project, inifile:
plugins: typeguard-2.7.1
collected 116 items / 108 deselected                                           

tests/test_cholesky.py::test_op_det[forward-cpu-A_shape0] PASSED         [ 12%]
tests/test_cholesky.py::test_op_det[forward-cpu-A_shape1] PASSED         [ 25%]
tests/test_cholesky.py::test_op_det[forward-cuda-A_shape0] PASSED        [ 37%]
tests/test_cholesky.py::test_op_det[forward-cuda-A_shape1] PASSED        [ 50%]
tests/test_cholesky.py::test_op_det[backward-cpu-A_shape0] PASSED        [ 62%]
tests/test_cholesky.py::test_op_det[backward-cpu-A_shape1] PASSED        [ 75%]
tests/test_cholesky.py::test_op_det[backward-cuda-A_shape0] PASSED       [ 87%]
tests/test_cholesky.py::test_op_det[backward-cuda-A_shape1] PASSED       [100%]

=

In [6]:
!python3 -m pytest -l -v -k "op_cholesky"

============================= test session starts ==============================
platform linux -- Python 3.8.16, pytest-3.6.4, py-1.11.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/project, inifile:
plugins: typeguard-2.7.1
collected 116 items / 108 deselected                                           

tests/test_cholesky.py::test_op_cholesky[forward-cpu-A_shape0] PASSED    [ 12%]
tests/test_cholesky.py::test_op_cholesky[forward-cpu-A_shape1] PASSED    [ 25%]
tests/test_cholesky.py::test_op_cholesky[forward-cuda-A_shape0] PASSED   [ 37%]
tests/test_cholesky.py::test_op_cholesky[forward-cuda-A_shape1] PASSED   [ 50%]
tests/test_cholesky.py::test_op_cholesky[backward-cpu-A_shape0] PASSED   [ 62%]
tests/test_cholesky.py::test_op_cholesky[backward-cpu-A_shape1] PASSED   [ 75%]
tests/test_cholesky.py::test_op_cholesky[backward-cuda-A_shape0] PASSED  [ 87%]
tests/test_cholesky.py::test_op_cholesky[backward-cuda-A_shape1] PASSED  [100%]

=